In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
import os
from tqdm import tqdm
import seaborn as sns
import math
import random

#import model library
import lightgbm as lgb
from lightgbm import LGBMRegressor
from xgboost import XGBRegressor
from catboost import CatBoostRegressor
from sklearn.metrics import roc_auc_score,mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
import warnings

warnings.filterwarnings('ignore')

In [ ]:
#Load data
datapath = Path('../input/ump-csv-parquet')
sample_test_path = Path('../input/ubiquant-market-prediction')
!ls $sample_test_path



In [ ]:
#train = pd.read_pickle('../input/ump-pickle-trainfile/train.pkl')
train = pd.read_parquet(datapath/'train.parquet')
train.drop(['row_id','time_id'],axis=1,inplace=True)
X = train.drop(['target'],axis=1)
y = train['target']
del train
xtrain,xvalid,ytrain,yvalid = train_test_split(X,y,test_size=0.1,random_state=42,shuffle=False)
print(xtrain.shape,xvalid.shape,ytrain.shape,yvalid.shape)
del X
del y


In [ ]:
#Model hyperparameter of XGboostRegressor
    #lgb parameters
params_lgb = {
    "task": "train",
    "boosting_type": "gbdt",
    "objective": "regression",
    'subsample': 0.95312,
    'learning_rate': 0.0011635,
    "max_depth": 3,
    "feature_fraction": 0.2256038826485174,
    "bagging_fraction": 0.7705303688019942,
    "min_child_samples": 290,
    "reg_alpha": 14.68267919457715,
    "reg_lambda": 66.156,
    "max_bin": 772,
    "min_data_per_group": 177,
    "bagging_freq": 1,
    "cat_smooth": 96,
    "cat_l2": 17,
    "verbosity": -1,
    'random_state':42,
    'n_estimators':15000,
    'colsample_bytree':0.1107
    }
    
lgb_train = lgb.Dataset(xtrain, ytrain)
lgb_val = lgb.Dataset(xvalid, yvalid)
    
model = lgb.train(params=params_lgb,
                      train_set=lgb_train,
                      valid_sets=lgb_val,
                      early_stopping_rounds=300,
                      verbose_eval=False)
    
preds_valid = model.predict(xvalid,num_iteration=model.best_iteration)
#test_predict = model.predict(xtest,num_iteration=model.best_iteration)

In [ ]:
import ubiquant
env = ubiquant.make_env()   # initialize the environment
iter_test = env.iter_test()    # an iterator which loops over the test set and sample submission
for (test_df, sample_prediction_df) in iter_test:
    test_df.drop(['row_id'],axis=1,inplace=True)
    pred = model.predict(test_df)
    sample_prediction_df['target'] = pred  # make your predictions here
    env.predict(sample_prediction_df)   # register your predictions